# Jina Search

This notebook provides a quick overview for getting started with Jina [tool](/docs/integrations/tools/). For detailed documentation of all Jina features and configurations head to the [API reference](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.jina_search.tool.JinaSearch.html).

## Overview

### Integration details

| Class | Package | Serializable | JS support |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [JinaSearch](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.jina_search.tool.JinaSearch.html) | [langchain-community](https://python.langchain.com/api_reference/community/) | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async | Return data | Pricing |
| :---: | :---: | :---: | :---: |
| ❌ | ❌ | URL, Snippet, Title, Page Content | 1M response tokens free | 


## Setup

The integration lives in the `langchain-community` package and was added in version `0.2.16`:

In [ ]:
%pip install --quiet -U "langchain-community>=0.2.16"

### Credentials

In [2]:
import getpass
import os

if not os.environ.get("JINA_API_KEY"):
    os.environ["JINA_API_KEY"] = getpass.getpass("Jina API key:\n")

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Instantiation

- TODO: Fill in instantiation params

Here we show how to instantiate an instance of the Jina tool, with 

In [2]:
from langchain_community.tools import JinaSearch

tool = JinaSearch()

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

In [4]:
print(tool.invoke({"query": "what is langgraph"})[:1000])

[{"title": "LangGraph", "link": "https://www.langchain.com/langgraph", "snippet": "<strong>LangGraph</strong> helps teams of all sizes, across all industries, from ambitious startups to established enterprises. \u201cLangChain is streets ahead with what they&#x27;ve put forward with <strong>LangGraph</strong>.", "content": "![Image 1](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/667b080e4b3ca12dc5d5d439_Langgraph%20UI-2.webp)\n\nControllable cognitive architecture for any task\n------------------------------------------------\n\nLangGraph's flexible API supports diverse control flows \u2013 single agent, multi-agent, hierarchical, sequential \u2013 and robustly handles realistic, complex scenarios.\n\nEnsure reliability with easy-to-add moderation and quality loops that prevent agents from veering off course.\n\n[See the docs](https://langchain-ai.github.io/langgraph/)\n\nDesigned for human-agent collaboration\n--------------------------------------\n\nWith built-in stat

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [6]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "what is langgraph"},
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)
print(tool_msg.content[:1000])

[{"title": "LangGraph Tutorial: What Is LangGraph and How to Use It?", "link": "https://www.datacamp.com/tutorial/langgraph-tutorial", "snippet": "<strong>LangGraph</strong> <strong>is</strong> a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner.", "content": "Imagine you're building a complex, multi-agent large language model (LLM) application. It's exciting, but it comes with challenges: managing the state of various agents, coordinating their interactions, and handling errors effectively. This is where LangGraph can help.\n\nLangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner.\n\nIt simplifies the development process by enabling the creation of cyclical graphs, which are essential for de

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [ ]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai")

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)


llm_with_tools = llm.bind_tools([tool])
llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("what's langgraph")

AIMessage(content="LangGraph is a library designed for building stateful, multi-actor applications with language models (LLMs). It is particularly useful for creating agent and multi-agent workflows. Compared to other LLM frameworks, LangGraph offers unique benefits such as cycles, controllability, and persistence. Here are some key points:\n\n1. **Stateful and Multi-Actor Applications**: LangGraph allows for the definition of flows involving cycles, essential for most agentic architectures. This is a significant differentiation from Directed Acyclic Graph (DAG)-based solutions.\n\n2. **Controllability**: The framework offers fine-grained control over both the flow and state of applications, which is crucial for creating reliable agents.\n\n3. **Persistence**: Built-in persistence is available, enabling advanced features like human-in-the-loop workflows and memory.\n\n4. **Human-in-the-Loop**: LangGraph supports interrupting graph execution for human approval or editing of the agent's 

## API reference

For detailed documentation of all Jina features and configurations head to the API reference: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.jina_search.tool.JinaSearch.html